In [57]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import matplotlib
import csv
matplotlib.rcParams['figure.figsize'] = (20, 10)

## Change the path to the extracted recording

In [184]:
path = "../1811calib"

In [185]:
pathL = os.path.join(path, '1')
pathR = os.path.join(path, '2')

In [201]:
CH_W = 7
CH_H = 5
EVERY_NTH_FRAME = 50
# Square size in mm 
SQUARE_SIZE = 30
# Termination criteria for refining the detected corners (taken from openCV docs as reasonable params)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


objp = np.zeros((CH_H*CH_W,3), np.float32)
objp[:,:2] = np.mgrid[0:CH_H,0:CH_W].T.reshape(-1,2)
objp *= SQUARE_SIZE

In [202]:

img_ptsL = []
img_ptsR = []
obj_pts = []
counter = 0

with open(os.path.join(path, 'match.csv')) as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        counter += 1
        if counter % EVERY_NTH_FRAME == 0:
            print(f'{pathL}{row[1]}.png')
            imgL = cv2.imread(os.path.join(pathL, f'{row[2]}.png'))
            imgR = cv2.imread(os.path.join(pathR, f'{row[1]}.png'))
            print(f'Time difference: {int(row[2]) - int(row[1])} ns')
            imgL_gray = cv2.imread(os.path.join(pathL, f'{row[2]}.png'),0)
            imgR_gray = cv2.imread(os.path.join(pathR, f'{row[1]}.png'),0)

            outputL = imgL.copy()
            outputR = imgR.copy()

            retR, cornersR =  cv2.findChessboardCorners(outputR,(CH_H,CH_W),None)
            retL, cornersL = cv2.findChessboardCorners(outputL,(CH_H,CH_W),None)

            if retR and retL:
                obj_pts.append(objp)
                cv2.cornerSubPix(imgR_gray,cornersR,(11,11),(-1,-1),criteria)
                cv2.cornerSubPix(imgL_gray,cornersL,(11,11),(-1,-1),criteria)
                cv2.drawChessboardCorners(outputR,(CH_H,CH_W),cornersR,retR)
                cv2.drawChessboardCorners(outputL,(CH_H,CH_W),cornersL,retL)

                img_ptsL.append(cornersL)
                img_ptsR.append(cornersR)


# Calibrating left camera
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(obj_pts,img_ptsL,imgL_gray.shape[::-1],None,None)

hL,wL= imgL_gray.shape[:2]
new_mtxL, roiL= cv2.getOptimalNewCameraMatrix(mtxL,distL,(wL,hL),1,(wL,hL))

# Calibrating right camera
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(obj_pts,img_ptsR,imgR_gray.shape[::-1],None,None)

hR,wR= imgR_gray.shape[:2]
new_mtxR, roiR= cv2.getOptimalNewCameraMatrix(mtxR,distR,(wR,hR),1,(wR,hR))

../1811calib/11637256585173648579.png
Time difference: -39250 ns
../1811calib/11637256586840049232.png
Time difference: -37712 ns
../1811calib/11637256588506446653.png
Time difference: -38865 ns
../1811calib/11637256590172849460.png
Time difference: -40173 ns
../1811calib/11637256591839247920.png
Time difference: -38673 ns
../1811calib/11637256593505649419.png
Time difference: -41788 ns
../1811calib/11637256595172045879.png
Time difference: -38865 ns
../1811calib/11637256596838443263.png
Time difference: -35019 ns
../1811calib/11637256598504846223.png
Time difference: -40557 ns
../1811calib/11637256600171244876.png
Time difference: -39289 ns
../1811calib/11637256601837645990.png
Time difference: -42019 ns
../1811calib/11637256603504039835.png
Time difference: -34712 ns
../1811calib/11637256605170447949.png
Time difference: -44788 ns
../1811calib/11637256606836842525.png
Time difference: -37865 ns
../1811calib/11637256608503242216.png
Time difference: -37596 ns
../1811calib/116372566101

In [203]:
np.savez("intrinsic_data_left", mtx=mtxL, dist=distL, rvecs=rvecsL, tvecs=tvecsL)
np.savez("intrinsic_data_right", mtx=mtxR, dist=distR, rvecs=rvecsR, tvecs=tvecsR)

# Stereo calibration

In [204]:
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(obj_pts, img_ptsL, img_ptsR, new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], criteria_stereo, flags)

In [205]:
rectify_scale=1
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR= cv2.stereoRectify(new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], Rot, Trns, rectify_scale,(0,0))

In [206]:
np.savez("extrinsic_data", Q=Q)

## stereoCalibrate retval

According to the docs, represents the final value of the re-projection error.


In [207]:
retS

0.767956792818179

In [208]:
Trns

array([[-99.57293254],
       [ -0.23377043],
       [ -0.37056768]])

 - **this translation value is off by a few mm from real-world situation (measured by a ruler), although the returned reproj error is not that big?**
 
 Should be something like: (-98, ~0, ~8)

In [75]:
Q

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -3.79180916e+02],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        -2.62139257e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.75456100e+02],
       [ 0.00000000e+00,  0.00000000e+00,  9.94764095e-03,
        -0.00000000e+00]])

In [11]:
Left_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxL, distL, rect_l, proj_mat_l,
                                             imgL_gray.shape[::-1], cv2.CV_16SC2)
Right_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxR, distR, rect_r, proj_mat_r,
                                              imgR_gray.shape[::-1], cv2.CV_16SC2)

print("Saving parameters ......")
cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("Left_Stereo_Map_x",Left_Stereo_Map[0])
cv_file.write("Left_Stereo_Map_y",Left_Stereo_Map[1])
cv_file.write("Right_Stereo_Map_x",Right_Stereo_Map[0])
cv_file.write("Right_Stereo_Map_y",Right_Stereo_Map[1])
cv_file.write("Trns",Trns)
cv_file.release()

Saving parameters ......
